# Introduction

**Overview:** Brief description of the problem, the dataset, and the main objectives of the project.

# Setup 

## Imports

In [ ]:
import pandas as pd
import numpy as np

## Environment Variables 
**Note**: Setting environment variables is optional, but it is recommended if you store sensitive information (such as API keys or database credentials) in a `.env` file. Using environment variables helps keep such information secure and separate from your codebase.

In [ ]:
# Imports
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Get API key from .env 
api_key = os.getenv("API_KEY")

# Get SQL database credentials from .env
sql_username = os.getenv("SQL_USERNAME")
sql_password = os.getenv("SQL_PASSWORD")

# Data Loading

## csv 

In [ ]:
# Load data from a csv file into a Pandas DataFrame
df = pd.read_csv("your_csv_file_here.csv")

## MySQL

In [ ]:
# Imports 
from sqlalchemy import create_engine

# Database info
mysql_host = "localhost"  # Default hostname for a MySQL server running locally
mysql_port = 3306  # Default port for MySQL
mysql_database_name = "your_mysql_database_name_here"
mysql_table_name = "your_mysql_table_name_here"

# Create an SQLAlchemy engine for interacting with the MySQL database
engine = create_engine(f"mysql+mysqlconnector://{sql_username}:{sql_password}@{mysql_host}:{mysql_port}/{mysql_database_name}")

# Load data from MySQL database into a Pandas DataFrame
with engine.connect() as connection:
    df = pd.read_sql(f"SELECT * FROM {mysql_table_name}", con=connection)

# Data Quality Checks

In [ ]:
# Show DataFrame info
df.info()

In [ ]:
# Show top five rows
df.head()

# Data Preprocessing

## Handling Duplicates

Duplicates based on all columns:

In [ ]:
# Diagnose duplicates 
df.duplicated().value_counts()

In [ ]:
# Remove duplicates
df = df.drop_duplicates().copy()

Duplicates based on specific columns, e.g. the ID column or a combination of columns:

In [ ]:
# Diagnose duplicates
df.duplicated(subset=["column_1", "column_2", "column_3"]).value_counts()

In [ ]:
# Remove duplicates
df = df.drop_duplicates(subset=["column_1", "column_2", "column_3"]).copy()

## Data Type Conversion

In [ ]:
# Convert column from str to int
df["int_column"] = df["str_column"].astype("Int32")

In [ ]:
# Convert column from str to datetime
df["datetime_column"] = pd.to_datetime(df["str_column"])

## Feature Extraction

Extract categorical feature from string column:

In [ ]:
# Function to extract a category from a string   
def extract_category_from_string(string):
    # Map categories to their corresponding list of keywords
    category_keywords_map = {
        "Category 1": ["Keyword 1", "Keyword 2", "Keyword 3"],
        "Category 2": ["Keyword 4", "Keyword 5", "Keyword 6"],
        "Category 3": ["Keyword 7", "Keyword 8", "Keyword 9"]
    }

    # Loop through each category and its associated keywords 
    for category, keywords in category_keywords_map.items():
        # Check if any keyword is present in the string
        if any(keyword in string for keyword in keywords):
            return category  # Return the category corresponding to the keyword
    return np.nan  # Return a missing value if no keyword matches

# Apply function on an existing string column to create a new categorical feature column
df["categorical_feature"] = df["str_column"].apply(extract_category_from_string)

# Show category frequencies
print(df["categorical_feature"].value_counts())

Extract numerical feature from string column:

In [ ]:
# Imports
import re

# Function to extract the first number in a string 
def extract_number_from_string(string):
    first_number = re.search(r"\b-?\d+([.,]\d+)?\b", string)  # searches for first integer or float (positive or negative; decimal separator "." or ",")
    if first_number:
        return float(first_number.group().replace(",", "."))  # Replace "," with "." as decimal separator  
    else:
        return np.nan  # Return a missing value if no number in string

# Apply function on an existing string column to create a new numerical feature column
df["numerical_feature"] = df["str_column"].apply(extract_number_from_string)

# Show descriptive statistics of numerical feature
df["numerical_feature"].describe()

Extract boolean feature from string column:

In [ ]:
# List of keywords to determine if the feature is present or absent
keywords = ["keyword 1", "keyword 2", "keyword 3"]

# Extract boolean feature column: True if any keyword is found in the string column
df["boolean_feature"] = df["str_column"].apply(lambda x: any(keyword.lower() in x.lower() for keyword in keywords))

# Show frequencies of boolean feature
print(df["boolean_feature"].value_counts())

## Handling Missing Values

### Imputation

Imputation for continuous column:

In [ ]:
# Descriptive statistics of numerical column
df["numerical_column"].describe()

In [ ]:
# Impute missing values with the median
median = df["numerical_column"].median()
df["numerical_column"] = df["numerical_column"].fillna(median)

Imputation for categorical column:

In [ ]:
# Frequencies of categorical column
df["categorical_column"].value_counts()

In [ ]:
# Impute missing values with the mode 
mode = df["categorical_column"].mode()[0]
df["categorical_column"] = df["categorical_column"].fillna(mode)

### Deletion

In [ ]:
# Delete rows where any column has a missing value 
df.dropna(inplace=True)

In [ ]:
# Delete rows where either column_1 or column_2 has a missing value 
df.dropna(subset=["column_1", "column_2"], how="any", inplace=True)

# Exploratory Data Analysis (EDA)

## Column Types

In [ ]:
# Show columns and their data types
df.info()

In [ ]:
# Define numerical, categorical and boolean columns
numerical_columns = ["numerical_column_1", "numerical_column_2", "numerical_column_3"]
categorical_columns = ["categorical_column_1", "categorical_column_2", "categorical_column_3"]
boolean_columns = ["boolean_column_1", "boolean_column_2", "boolean_column_3"]

# Train-Validation-Test Split

# Model Training